# First Step: As always, make sure we can read the data!
I usually just use the top block to make sure I can actually read the data and install all the libraries beforehand since it can be a bit of a mess/hard to read doing this stuff later into the notebook! Here I'm just making sure that I can actually access the dataset for the most part (as well as seeing if there are any problems with the current Python environment/kernel on the side...)

In [7]:
# Install some useful packages for data analysis and visualization
!pip install pandas --break-system-packages
!pip install matplotlib --break-system-packages
!pip install seaborn --break-system-packages
!pip install scikit-learn --break-system-packages
!pip install tensorflow --break-system-packages
!pip install keras --break-system-packages


from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

# Load the dataset
data = pd.read_csv('FoodX.csv')

# Display the first few rows of the dataset
data.head()

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


,Year,Major,University,Time,Order
0,Year 2,Physics,Indiana State University,12,Fried Catfish Basket
1,Year 3,Chemistry,Ball State University,14,Sugar Cream Pie
2,Year 3,Chemistry,Butler University,12,Indiana Pork Chili
3,Year 2,Biology,Indiana State University,11,Fried Catfish Basket
4,Year 3,Business Administration,Butler University,12,Indiana Corn on the Cob (brushed with garlic b...


# Choosing a model

Since this is a classifying task, I'm gonna try using Decision Trees and Random Forests. This is because of their efficiencies in these kind of tasks and also their ease of use in SKLearn

# Random Forest Implementation

In [42]:
# Data preprocessing
# Encode categorical variables using LabelEncoder
label_encoder = LabelEncoder()
data['Year'] = label_encoder.fit_transform(data['Year'])
data['Major'] = label_encoder.fit_transform(data['Major'])
data['University'] = label_encoder.fit_transform(data['University'])
data['Time'] = label_encoder.fit_transform(data['Time'])
data['Order'] = label_encoder.fit_transform(data['Order'])

# Split the data into features (X) and target (y)
X = data.drop('Order', axis=1) 
y = data['Order']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)

# Create and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)

# Make predictions on new data
y_pred = rf_model.predict(X_test)

# Evaluate the model (you can use different metrics depending on your task)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.696


# Let's see if we can juice it some more!

Well that accuracy is already kind of nice! Let's try tuning some of those parameters to see if we can get even better results

In [9]:
# Define a parameter grid for Grid Search
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2', None]
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best parameters and estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Evaluate the best model on the test data
accuracy = best_estimator.score(X_test, y_test)
print(f"Best Model Parameters: {best_params}")
print(f"Accuracy on test data with best model: {accuracy:.2f}")

Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Model Parameters: {'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5}
Accuracy on test data with best model: 0.69


# Hmm

Well trying to tune the parameters led me on a wild goose chase... Let's just try using simple decision trees to categorize these potential orders, the main reason being that finding something in decision trees which provides better accuracy should directly transfer over to Random Forests and training decision trees takes a whole lot less time!

# Decision Tree Implementation

Let's see how it fairs against the Random Forest implementation

In [46]:
# Create a Decision Tree model and fit it to the training data
decision_tree = DecisionTreeClassifier(random_state=1)
decision_tree.fit(X_train, y_train)

# Evaluate the model's performance on the test data
accuracy = decision_tree.score(X_test, y_test)
print(f"Accuracy on test data: {accuracy:.2f}")

Accuracy on test data: 0.59


# Celebrate!

Hey look at that, we got about the same accuracy as the Random Forest implementation with a *lot* less compute. Let's try and see how far we can go with a single decision tree.

In [50]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)

# Create a Decision Tree classifier
decision_tree = DecisionTreeClassifier(random_state=1)

# Let's make a range for floats to test a wider range of potential parameters
def frange(start, stop, step):
    res, n = start, 1
    while res < stop:
        yield res
        res = start + n * step
        n += 1

# Define a parameter grid for Grid Search
param_grid = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'min_samples_leaf': range(1,5),
    'class_weight': [None, 'balanced'],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes':range(100,500,100),
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=decision_tree, param_grid=param_grid, scoring='accuracy', cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best parameters and estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

# Evaluate the best model on the test data
accuracy = best_estimator.score(X_test, y_test)
print(f"Best Model Parameters: {best_params}")
print(f"Accuracy on test data with best model: {accuracy:.2f}")

Fitting 5 folds for each of 2784 candidates, totalling 13920 fits
Best Model Parameters: {'class_weight': 'balanced', 'criterion': 'gini', 'max_features': 4, 'max_leaf_nodes': 400, 'min_samples_leaf': 1}
Accuracy on test data with best model: 0.62


# So sad...

Looks like the best we're gonna get out of a decision trees is around ~62% accuracy.

As a last hoorah, I'm going to try and implement a very simple neural network afterwards using keras and tensorflow to see if it can do any better than the tree based models; most likely it won't though!

In [17]:
# Data preprocessing
# Encode categorical variables using LabelEncoder
label_encoder = LabelEncoder()
data['Year'] = label_encoder.fit_transform(data['Year'])
data['Major'] = label_encoder.fit_transform(data['Major'])
data['University'] = label_encoder.fit_transform(data['University'])
data['Time'] = label_encoder.fit_transform(data['Time'])
data['Order'] = label_encoder.fit_transform(data['Order'])

# Split the data into features (X) and target (y)
X = data.drop('Order', axis=1)  # Assuming 'Major' is the target variable
y = data['Order']

# One-hot encode the target variable (time of the food orders)
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=1)

# Create a simple feedforward neural network
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(y_train.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to the training data
model.fit(X_train, y_train, epochs=2000, batch_size=1024, verbose=1)

# Evaluate the model on the test data
accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
print(f"Accuracy on test data with neural network: {accuracy:.2f}")


Epoch 1/2000
5/5 [==============================] - 1s 4ms/step - loss: 2.3220 - accuracy: 0.1225
Epoch 2/2000
5/5 [==============================] - 0s 4ms/step - loss: 2.2367 - accuracy: 0.1518
Epoch 3/2000
5/5 [==============================] - 0s 4ms/step - loss: 2.1834 - accuracy: 0.1884
Epoch 4/2000
5/5 [==============================] - 0s 4ms/step - loss: 2.1366 - accuracy: 0.2286
Epoch 5/2000
5/5 [==============================] - 0s 4ms/step - loss: 2.0990 - accuracy: 0.2389
Epoch 6/2000
5/5 [==============================] - 0s 4ms/step - loss: 2.0632 - accuracy: 0.2695
Epoch 7/2000
5/5 [==============================] - 0s 3ms/step - loss: 2.0316 - accuracy: 0.3057
Epoch 8/2000
5/5 [==============================] - 0s 3ms/step - loss: 2.0040 - accuracy: 0.3091
Epoch 9/2000
5/5 [==============================] - 0s 3ms/step - loss: 1.9770 - accuracy: 0.3196
Epoch 10/2000
5/5 [==============================] - 0s 4ms/step - loss: 1.9515 - accuracy: 0.3467
Epoch 11/2000
5/5 [

# Conclusions

So we get the best accuracy from the Random Forest implementation which makes sense considering Decision Trees perform well in this task and Random Forest just makes a collection of these. I think with more data, however, that the neural network implementation would be better with more data and higher dimensional data.

For scaling, I would recommend the single Decision Tree just because of how few amount of resources it uses and how well it performs comparatively, other than that the obvious choice is just to use Random Forest for the higher accuracy in predicting orders!